In [3]:
import shutil
from pathlib import Path

import numpy as np
import pandas as pd
from tqdm import tqdm

from ultralytics.utils.downloads import download
from ultralytics.utils.ops import xyxy2xywh

# Download
dir = Path('F:\mx-projects\shelf-products\code\model-training\yolov8sku110k\dataset')  # dataset root dir
parent = Path(dir.parent)  # download dir
urls = ['http://trax-geometry.s3.amazonaws.com/cvpr_challenge/SKU110K_fixed.tar.gz']
download(urls, dir=parent)

# Rename directories
if dir.exists():
  shutil.rmtree(dir)
(parent / 'SKU110K_fixed').rename(dir)  # rename dir
(dir / 'labels').mkdir(parents=True, exist_ok=True)  # create labels dir

# Convert labels
names = 'image', 'x1', 'y1', 'x2', 'y2', 'class', 'image_width', 'image_height'  # column names
for d in 'annotations_train.csv', 'annotations_val.csv', 'annotations_test.csv':
  x = pd.read_csv(dir / 'annotations' / d, names=names).values  # annotations
  images, unique_images = x[:, 0], np.unique(x[:, 0])
  with open((dir / d).with_suffix('.txt').__str__().replace('annotations_', ''), 'w') as f:
      f.writelines(f'./images/{s}\n' for s in unique_images)
  for im in tqdm(unique_images, desc=f'Converting {dir / d}'):
      cls = 0  # single-class dataset
      with open((dir / 'labels' / im).with_suffix('.txt'), 'a') as f:
          for r in x[images == im]:
              w, h = r[6], r[7]  # image width, height
              xywh = xyxy2xywh(np.array([[r[1] / w, r[2] / h, r[3] / w, r[4] / h]]))[0]  # instance
              f.write(f"{cls} {xywh[0]:.5f} {xywh[1]:.5f} {xywh[2]:.5f} {xywh[3]:.5f}\n")  # write label

 10%|█         | 1.19G/11.4G [34:09<4:52:48, 622kB/s]  


KeyboardInterrupt: 